In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
req_file = scaler.fit_transform(np.array(req_file))

training_size = math.floor(len(req_file)*0.80)
test_size = len(req_file)-training_size
train_data, test_data = req_file[0:training_size, :], req_file[training_size:len(req_file), :]

# for cluster 7 change lookback = 8
X_train, Y_train = createDataset(train_data,8)
X_test, Y_test = createDataset(test_data, 8)

from tensorflow import keras
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import activations

model = keras.models.Sequential()
# for cluster 7 change lookback = 8
model.add(layers.LSTM(32, input_shape=(8, 2)))
model.add(layers.Dense(2))
model.add(layers.Activation(activations.relu))
model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

#early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1,patience = 25)
model.summary()
history=model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_test, Y_test))
#model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_test, Y_test))

test_predict = model.predict(X_test)
train_predict = model.predict(X_train)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_test, test_predict))

from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_train, train_predict))

# for cluster 7 change lookback = 8
look_back = 8
trainPredictPlot = np.empty_like(req_file)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

testPredictPlot = np.empty_like(req_file)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(req_file)-1, :] = test_predict

names = "cluster{}".format(cluster)+"---"+"{}".format(route)+"_"+"{}".format(direction)+"_"+"{}".format(stop)
accura = [history.history['val_accuracy'][0],history.history['val_accuracy'][len(history.history['val_accuracy'])-1]]
accu = "{:.4f} , {:.4f}".format(*accura)


plt.plot(scaler.inverse_transform(req_file)[:, 0])
plt.plot(trainPredictPlot[:, 0])
plt.plot(testPredictPlot[:, 0])
# plt.legend(['Actual', 'Train Prediction', 'Test Predict'], ncol=3, loc=9, handletextpad=0.5)
plt.ylabel('Passenger Count')
plt.xlabel('Trips (From december to march)')
plt.title("cluster{}".format(cluster)+"_"+"route{}".format(route)+"_"+"direction{}".format(direction)+"_"+"stop{}".format(stop)
          +"_"+accu)
plt.title("Cluster no{}".format(cluster))
plt.savefig("cluster_minute_new{}".format(cluster)+"---"+"{}".format(route)+"_"+"{}".format(direction)+"_"+"{}".format(stop))
#plt.savefig('7_0_12_predictions(cluster_7)_minute.pdf', dpi=300)  
#plt.show()

figures.append(plt.figure())